In [ ]:
from eccodes import codes_bufr_new_from_file, codes_get, codes_get_array, codes_release

def bufr_to_table(bufr_file, variables=None):
    """
    Parse BUFR sounding/profile into a table (list of dicts).
    Each row = one level.
    """

    if variables is None:
        variables = [
            "pressure",
            "airTemperature",
            "dewpointTemperature",
            "windSpeed",
            "windDirection",
            "nonCoordinateGeopotentialHeight",
            "timePeriod"
        ]

    table = []

    with open(bufr_file, "rb") as f:
        while True:
            bufr = codes_bufr_new_from_file(f)
            if bufr is None:
                break

            codes_get(bufr, "unpack")

            # fetch arrays for each variable
            arrays = {}
            maxlen = 0
            for var in variables:
                try:
                    vals = codes_get_array(bufr, var)
                    arrays[var] = vals
                    if len(vals) > maxlen:
                        maxlen = len(vals)
                except Exception:
                    arrays[var] = []
            
            # align arrays by index
            for i in range(maxlen):
                row = {}
                for var in variables:
                    if i < len(arrays[var]):
                        row[var] = arrays[var][i]
                    else:
                        row[var] = None  # missing if shorter
                table.append(row)

            codes_release(bufr)

    return table

In [ ]:
# define sample file
bufr_file = "/root/sample-data/WIGOS_0-20000-0-78583_20250728T230800.bufr4"

rows = bufr_to_table(bufr_file)
print("Decoded", len(rows), "rows")
for r in rows[:20]:  # print first 20 rows
    print(r)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_sounding(rows):
    # Extract data from list of dicts into arrays
    p = []
    T = []
    Td = []
    wspd = []
    wdir = []

    for r in rows:
        if (r.get("pressure") is None or 
            r.get("airTemperature") is None or 
            r.get("dewpointTemperature") is None):
            continue

        p.append(r["pressure"] / 100.0)  # Pa → hPa
        T.append(r["airTemperature"] - 273.15)   # K → °C
        Td.append(r["dewpointTemperature"] - 273.15)
        wspd.append(r.get("windSpeed", np.nan))
        wdir.append(r.get("windDirection", np.nan))

    p = np.array(p)
    T = np.array(T)
    Td = np.array(Td)
    wspd = np.array(wspd)
    wdir = np.array(wdir)

    # Compute wind components for barbs
    u = -wspd * np.sin(np.deg2rad(wdir))
    v = -wspd * np.cos(np.deg2rad(wdir))

    # Create figure
    fig, ax = plt.subplots(figsize=(6, 9))

    # Plot Temperature and Dewpoint vs Pressure
    ax.plot(T, p, 'r', label="Temperature")
    ax.plot(Td, p, 'g', label="Dewpoint")

    # Add wind barbs every Nth level
    skip = max(1, len(p)//30)  # thin out if too many levels
    x_barb = np.full_like(p[::skip], fill_value=35.0)  # fixed x position for barbs
    ax.barbs(x_barb, p[::skip], u[::skip], v[::skip],
             length=6, pivot='middle')

    # Pressure axis (log scale, inverted)
    ax.set_yscale("log")
    ax.set_ylim(1050, 100)
    ax.invert_yaxis()
    ax.set_yticks([1000, 850, 700, 500, 300, 200, 100])
    ax.get_yaxis().set_major_formatter(plt.ScalarFormatter())

    ax.set_xlabel("Temperature (°C)")
    ax.set_ylabel("Pressure (hPa)")
    ax.set_title("Upper-Air Sounding")
    ax.legend()

    plt.show()


# Example usage:
#rows = bufr_to_table("your_file.bufr")
plot_sounding(rows)